<a href="https://colab.research.google.com/github/marcellobenedetti92/machine-learning-fondamenti/blob/main/progetto_finale_codice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Il molto bassa è 25-30? In realta' va pure bene, cioè voi dovete considerare il vantaggio economico che avrebbe l'azienda ad usare il vostro modello piuttosto che ad andare a caso. In questo caso è importante non perdersi dei potenziali acquirenti, quindi la recall è più importante, e nel modello è molto alta. Se la precision è più bassa vuol dire che i Sales faranno più chiamate a vuoto, quindi ci potrebbe essere uno spreco economico inteso come ora lavoro del sale, con una precision del 25% vuol dire che 1/4 dei chiamati non è in realta' interessato, però andando a caso questo numero sarebbe stato meno di 1/8 (distribuzione delle classi), quindi meno della metà. Quindi in realta' c'è un risparmio economico non indifferente.*

# Modello di classificazione binaria

### Import

Import dei moduli funzionali

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import matplotlib.pyplot as plt
import seaborn as sns

Lettura csv precedentemente caricato nello spazio virtuale di Colab e salvataggio in un dataframe

In [ ]:
df_train = pd.read_csv("train.csv")

### Pre-processing

#### Drop id columns

Droppo la colonna relativa all'id perché popolata da valori unici, quindi informazione non rilevante ai fini del modello

In [ ]:
df_train = df_train.drop("id", axis=1)

#### Features encoding

##### One Hot Encoding per variabili categoriali

In [ ]:
# per le classi maschio e femmina del SET DI TRAIN droppando la prima colonna perché non aggiunge nessuna info al modello
df_train = pd.get_dummies(df_train, columns=["Gender"], drop_first=True)
df_train.head()

In [ ]:
# per le classi di Region_Code del SET DI TRAIN
df_train = pd.get_dummies(df_train, columns=["Region_Code"])
df_train.head()

In [ ]:
# per le classi Policy_Sales_Channel del SET DI TRAIN
df_train = pd.get_dummies(df_train, columns=["Policy_Sales_Channel"])
df_train.head()

##### Label encoding per variabili ordinali

In [ ]:
# conteggio valori per le classi di Vehicle_Age sul set di train
counts_train = df_train["Vehicle_Age"].value_counts()
counts_train

In [ ]:
# sostituzione valori di Vehicle_Age sul set di train
age_mapping_train = {"< 1 Year":1, "1-2 Year":2, "> 2 Years":3}
df_train["Vehicle_Age"] = df_train["Vehicle_Age"].map(age_mapping_train)
df_train.head()

In [ ]:
# codifica delle osservazioni della feature Vehicle_Damage a 0|1 sul set di train

# stavolta con sklearn

# inizializzazione del Label Encoder
le = LabelEncoder()

# applicazione della trasformazione
df_train["Vehicle_Damage"] = le.fit_transform(df_train["Vehicle_Damage"])
df_train.head()

### Modello

#### Creazione

In [ ]:
# estrazione array con le features
X = df_train.drop("Response",axis = 1).values
# e con il target
y = df_train["Response"].values

In [ ]:
# creazione del set di test con il 30% dei valori del set di partenza
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [ ]:
# sommario della distribuzione in classi per il set di train
print(Counter(y_train))
# set sbilanciato

# FEEDBACK DEL COACH GULLO
# per prassi tali verifiche vanno fatte sul dataset intero,
# non sul train set, anche se in questo caso c'è poca differenza
# dato che la funzione train_test_split tende a mantenere la distribuzione delle classi presente nal dataset intero

#### Oversampling

In [ ]:
# PER IL SECONDO MODELLO

# definizione strategia di Oversampling
over = RandomOverSampler(sampling_strategy=0.25)
# fit e applicazione della trasformazione
X_train_ss25, y_train_ss25 = over.fit_resample(X_train, y_train)
# check ulteriore sommario della distribuzione delle classi
print(Counter(y_train_ss25))

In [ ]:
# PER IL TERZO MODELLO

# definizione strategia di Oversampling
over = RandomOverSampler(sampling_strategy=0.5)
# fit e applicazione della trasformazione
X_train_ss50, y_train_ss50 = over.fit_resample(X_train, y_train)
# check ulteriore sommario della distribuzione delle classi
print(Counter(y_train_ss50))

In [ ]:
# PER IL QUARTO MODELLO

# definizione strategia di Oversampling
over = RandomOverSampler(sampling_strategy=0.75)
# fit e applicazione della trasformazione
X_train_ss75, y_train_ss75 = over.fit_resample(X_train, y_train)
# check ulteriore sommario della distribuzione delle classi
print(Counter(y_train_ss75))

In [ ]:
# PER IL QUINTO MODELLO

# definizione strategia di Oversampling
over = RandomOverSampler(sampling_strategy=1)
# fit e applicazione della trasformazione
X_train_ss1, y_train_ss1 = over.fit_resample(X_train, y_train)
# check ulteriore sommario della distribuzione delle classi
print(Counter(y_train_ss1))

#### Standard Scaler e Fit del modello

In [ ]:
ss = StandardScaler()

In [ ]:
#COACH: Con ogni fit_transform vai a sovrascrivere i valori di media e deviazione standard utilizzati per la trasformazione
# questo vuol dire che X_test verrà standardizzato come X_train_ss1, di conseguenza non è corretto utilizzarlo anche per valutare gli altri modelli
# avresti dovuto creare un test set per ogni train set, oppure STANDARDIZZARE E ADDESTRARE UN MODELLO PER VOLTA

#IO: ho diviso in blocchi di codice separati così da rispettare quanto sopra in maiuscolo

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
X_train_ss25 = ss.fit_transform(X_train_ss25)
X_test = ss.transform(X_test)

In [ ]:
X_train_ss50 = ss.fit_transform(X_train_ss50)
X_test = ss.transform(X_test)

In [ ]:
X_train_ss75 = ss.fit_transform(X_train_ss75)
X_test = ss.transform(X_test)

In [ ]:
X_train_ss1 = ss.fit_transform(X_train_ss1) 
X_test = ss.transform(X_test)

In [ ]:
lr = LogisticRegression()

In [ ]:
# anche qui stessa cosa di sopra al commento IO

lr.fit(X_train, y_train)

In [ ]:
lr.fit(X_train_ss25, y_train_ss25)

In [ ]:
lr.fit(X_train_ss50, y_train_ss50)

In [ ]:
lr.fit(X_train_ss75, y_train_ss75)

In [ ]:
lr.fit(X_train_ss1, y_train_ss1)

#### Valutazione modello

In [ ]:
y_pred_train = lr.predict(X_train)
y_proba_train = lr.predict_proba(X_train)

y_pred_test = lr.predict(X_test)
y_proba_test = lr.predict_proba(X_test)

In [ ]:
y_pred_train_ss25 = lr.predict(X_train_ss25)
y_proba_train_ss25 = lr.predict_proba(X_train_ss25)

y_pred_test_ss25 = lr.predict(X_test)
y_proba_test_ss25 = lr.predict_proba(X_test)

In [ ]:
y_pred_train_ss50 = lr.predict(X_train_ss50)
y_proba_train_ss50 = lr.predict_proba(X_train_ss50)

y_pred_test_ss50 = lr.predict(X_test)
y_proba_test_ss50 = lr.predict_proba(X_test)

In [ ]:
y_pred_train_ss75 = lr.predict(X_train_ss75)
y_proba_train_ss75 = lr.predict_proba(X_train_ss75)

y_pred_test_ss75 = lr.predict(X_test)
y_proba_test_ss75 = lr.predict_proba(X_test)

In [ ]:
y_pred_train_ss1 = lr.predict(X_train_ss1)
y_proba_train_ss1 = lr.predict_proba(X_train_ss1)

y_pred_test_ss1 = lr.predict(X_test)
y_proba_test_ss1 = lr.predict_proba(X_test)

##### Modello senza Oversampling

In [ ]:
print(classification_report(y_train, y_pred_train))

In [ ]:
print(classification_report(y_test, y_pred_test))

##### Modello con Oversampling 0.25

In [ ]:
print(classification_report(y_train_ss25, y_pred_train_ss25))

In [ ]:
print(classification_report(y_test, y_pred_test_ss25))

##### Modello con Oversampling 0.5

In [ ]:
print(classification_report(y_train_ss50, y_pred_train_ss50))

In [ ]:
print(classification_report(y_test, y_pred_test_ss50))

##### Modello con Oversampling 0.75

In [ ]:
print(classification_report(y_train_ss75, y_pred_train_ss75))

In [ ]:
print(classification_report(y_test, y_pred_test_ss75))

##### Modello con Oversampling 1

In [ ]:
print(classification_report(y_train_ss1, y_pred_train_ss1))

In [ ]:
print(classification_report(y_test, y_pred_test_ss1))